## Práctica 2. Procesamiento de lenguaje natural

### Parte 1. Análisis de sentimientos

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
with open("yelp_labelled.txt","r") as text_file:
    lines_org = text_file.read().split('\n')
len(lines_org)

1001

In [3]:
lines = [line.split("\t") for line in lines_org if len(line.split("\t"))==2 and line.split("\t")[1]!='']
len(lines)

1000

In [4]:
data_line = np.array(lines)
text = data_line[:,0]
target = data_line[:,1]

In [5]:
data = {'Texto':text,'Target':target}

In [6]:
df = pd.DataFrame(data=data)
df.head()

,Texto,Target
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [7]:
df.keys()

Index(['Texto', 'Target'], dtype='object')

### Apartado a)

#### Configura una partición train-test usando el 75% de los datos para entrenamiento y el 25% restante para test.

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Texto'], df['Target'], test_size=0.25, random_state=333)


#### Vamos a estudiar varias representaciones de bolsa de palabras, pero todas ellas utilizarán countVectorizer con el diccionario que se crea a partir de los términos del propio corpus y la lista de palabras vacías (stop_words) que proporciona sklearn para el inglés. Las 4 posibilidades que estudiaremos surgen de combinar los siguientes 2 parámetros:

    - Bolsa de palabras binaria y bolsa de palabras con TF/IDF (parámetro binary).
    
    - Usando un rango de n-gramas de (1,1) y de (1,2) (parámetro ngram_range).

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [10]:
vectorizer = CountVectorizer(stop_words='english')
# Tomamos los textos del conjunto de entrenamiento y los transformamos en 
# una matriz de datos (palabras) según el diccionario estándar
train_vector_data=vectorizer.fit_transform(X_train)

In [28]:
print(train_vector_data)

  (0, 440)	1
  (0, 448)	1
  (0, 516)	1
  (0, 859)	1
  (0, 1074)	1
  (1, 175)	1
  (1, 710)	1
  (1, 1412)	1
  (2, 86)	1
  (2, 506)	1
  (2, 744)	1
  (2, 1152)	1
  (2, 1256)	1
  (3, 958)	1
  (3, 1244)	1
  (3, 1271)	1
  (4, 101)	1
  (4, 587)	1
  (4, 732)	1
  (5, 615)	1
  (5, 1087)	1
  (6, 269)	1
  (6, 940)	1
  (6, 1226)	1
  (6, 1338)	1
  :	:
  (744, 120)	1
  (744, 803)	1
  (744, 1083)	1
  (745, 127)	1
  (745, 749)	1
  (745, 772)	1
  (745, 1197)	1
  (745, 1434)	1
  (746, 360)	1
  (747, 423)	1
  (747, 521)	1
  (747, 744)	1
  (747, 755)	1
  (747, 856)	1
  (747, 860)	1
  (747, 900)	1
  (747, 1000)	1
  (747, 1149)	1
  (747, 1223)	1
  (748, 506)	1
  (748, 553)	1
  (748, 1412)	1
  (749, 447)	1
  (749, 990)	1
  (749, 1120)	1


In [29]:
feature_names = vectorizer.get_feature_names()

print(len(feature_names))
print(feature_names)
#print(train_vector_data[10])

1472
['00', '10', '100', '12', '15', '17', '1979', '20', '2007', '23', '30', '35', '40', '40min', '45', '4ths', '5lb', '85', '90', 'absolutely', 'absolutley', 'accomodate', 'accordingly', 'accountant', 'ache', 'acknowledged', 'actual', 'actually', 'added', 'afternoon', 'ago', 'airline', 'albondigas', 'allergy', 'almonds', 'amazing', 'ambiance', 'ambience', 'andddd', 'annoying', 'anymore', 'anytime', 'anyways', 'apologize', 'apology', 'app', 'appalling', 'appealing', 'appetite', 'appetizer', 'appetizers', 'apple', 'approval', 'area', 'arepas', 'aria', 'array', 'arrived', 'arriving', 'article', 'ask', 'asked', 'asking', 'assure', 'ate', 'atmosphere', 'atrocious', 'attached', 'attentive', 'attitudes', 'auju', 'authentic', 'average', 'avoid', 'avoided', 'away', 'awesome', 'awful', 'awkward', 'awkwardly', 'ayce', 'az', 'baba', 'baby', 'bachi', 'bacon', 'bad', 'bagels', 'bakery', 'baklava', 'ball', 'bamboo', 'banana', 'bank', 'bar', 'barely', 'bargain', 'bartender', 'bartenders', 'baseball',

In [30]:
import numpy as np
import numpy.ma as ma

def write_terms (feature_names, data, vector_data, index):
    '''
    Escribe los términos presentes en un mensaje representado como bolsa de palabras.
    
    - feature_names: terminos usados para vectorizar
    - data: lista de mensajes original (si data==None no se muestra el mensaje original)
    - vector_data: matriz (dispersa) de mensaje vectorizados
    - index: posición del mensaje a mostrar
    '''
    # máscara para seleccionar sólo el mensaje en posición index
    mask=vector_data[index,:]>0
    
    # términos que aparecen en ese mensaje vectorizado
    terminos = ma.array(feature_names, mask = ~(mask[0].toarray()))
    
    # mostrar mensaje original
    if data:
        print('Mensaje', index, ':', data[index])
    
    # mostrar términos que aparecen en el mensaje vectorizado
    print('Mensaje', index, 'vectorizado:', terminos.compressed(),'\n')

In [31]:
#write_terms(feature_names, train_data.data, train_vector_data, 10)
write_terms(feature_names, None, train_vector_data, 0)
write_terms(feature_names, None, train_vector_data, 10)

write_terms(feature_names, None, train_vector_data, 100)
write_terms(feature_names, None, train_vector_data, 200)

Mensaje 0 vectorizado: ['excellent' 'experienced' 'frenchman' 'new' 'restaurant'] 

Mensaje 10 vectorizado: ['caballero' 'recently' 'tried' 'week'] 

Mensaje 100 vectorizado: ['moist' 'shrimp' 'tender'] 

Mensaje 200 vectorizado: ['disgusted' 'hair' 'human' 'pretty' 'sure'] 



In [32]:
# Calculamos el valor TF-IDF 
tfidfer = TfidfTransformer()
train_preprocessed = tfidfer.fit_transform(train_vector_data)

In [33]:
#write_terms(feature_names, train_data.data, train_vector_data, 10)
write_terms(feature_names, None, train_vector_data, 10)

write_terms(feature_names, None, train_vector_data, 100)

Mensaje 10 vectorizado: ['caballero' 'recently' 'tried' 'week'] 

Mensaje 100 vectorizado: ['moist' 'shrimp' 'tender'] 



In [34]:
# Tomamos los textos del conjunto de test y los transformamos en una matriz
# de palabras. Al usar "transform" toma como referencia únicamente las palabras
# encontradas en el conjunto de entrenamiento
test_vector_data=vectorizer.transform(X_test)
# Calculamos el valor TF-IDF 
# Al usar "transform" toma como IDF el del conjunto de entrenamiento 
test_preprocessed=tfidfer.transform(test_vector_data)

In [35]:
print(test_vector_data)

  (0, 241)	1
  (0, 517)	1
  (0, 1118)	1
  (0, 1219)	1
  (0, 1412)	1
  (1, 331)	1
  (1, 626)	1
  (1, 748)	1
  (1, 1210)	1
  (1, 1332)	1
  (2, 95)	1
  (2, 159)	1
  (2, 506)	1
  (2, 772)	1
  (2, 1149)	1
  (2, 1175)	1
  (2, 1400)	1
  (3, 1304)	1
  (5, 506)	1
  (5, 567)	1
  (6, 409)	1
  (6, 565)	1
  (6, 737)	1
  (6, 877)	1
  (6, 958)	1
  :	:
  (242, 1152)	1
  (243, 36)	1
  (243, 88)	1
  (243, 180)	1
  (243, 230)	1
  (243, 521)	1
  (243, 764)	1
  (243, 860)	1
  (243, 1214)	1
  (244, 972)	1
  (245, 553)	1
  (245, 813)	1
  (245, 1246)	1
  (246, 121)	1
  (246, 1214)	1
  (246, 1445)	1
  (247, 703)	1
  (247, 737)	1
  (247, 985)	1
  (247, 1293)	1
  (247, 1310)	1
  (248, 553)	1
  (248, 1310)	1
  (249, 1189)	1
  (249, 1311)	1


¿¿¿¿ n-gramas ?????



    
#### Para cada una de esas 4 combinaciones entrenaremos dos clasificadores:

     1. Un clasificador naive bayes, eligiendo el más adecuado para cada caso.
     
     2. Un árbol de decisión buscando un valor óptimo para uno de los siguientes parámetros para que se maximice la tasa de aciertos en el conjunto de test: max_depth, min_samples_leaf o max_leaf_nodes (siempre el mismo).
     


In [36]:
from sklearn.naive_bayes import MultinomialNB

mnb_classifier = MultinomialNB()

mnb_classifier.fit(train_preprocessed, y_train)

mnb_train_predictions = mnb_classifier.predict(train_preprocessed)
mnb_test_predictions = mnb_classifier.predict(test_preprocessed)

print("Multinomial Naive Bayes, porcentaje de aciertos en entrenamiento:", np.mean(mnb_train_predictions == y_train))
print("Multinomial Naive Bayes, porcentaje de aciertos en test:", np.mean(mnb_test_predictions == y_test))

Multinomial Naive Bayes, porcentaje de aciertos en entrenamiento: 0.968
Multinomial Naive Bayes, porcentaje de aciertos en test: 0.744


In [37]:
from sklearn import tree
import numpy as np

# Creamos el clasificador con los valores por defecto
tree_classifier = tree.DecisionTreeClassifier()
tree_classifier.fit(train_preprocessed, y_train)

tree_train_predictions = tree_classifier.predict(train_preprocessed)
tree_test_predictions = tree_classifier.predict(test_preprocessed)

print("Árbol, porcentaje de aciertos en entrenamiento:", np.mean(tree_train_predictions == y_train))
print("Árbol, porcentaje de aciertos en test:", np.mean(tree_test_predictions == y_test))

Árbol, porcentaje de aciertos en entrenamiento: 0.9973333333333333
Árbol, porcentaje de aciertos en test: 0.676


#### Analiza la tasa de aciertos de entrenamiento y test de los 2 clasificadores en las 4 representaciones de bolsa de palabras (8 configuraciones en total) y contesta a las siguientes preguntas:

    - ¿Hay un clasificador que sea superior al otro? ¿por qué crees que sucede?
    
    - Para cada clasificador, ¿tiene un efecto positivo el añadir “complejidad” a la vectorización? Es decir, añadir bigramas y añadir tf-idf. ¿Por qué crees que sucede este efecto positivo o la falta del mismo? 
    


Los resultados que obtenemos con el arbol de decisión son mejores que los obtenidos con el clasificador de naive bayes. MIRAR ARBOLES DE DECISION Y NAIVE BAYES (para contestar por que creemos que sucede eso)

#### Selecciona el mejor árbol de decisión y obtén las 25 variables con más poder discriminante:
    
    - ¿Predominan más las palabras de uno u otro sentimiento? ¿por qué? ¿hay ruido? 
    


In [38]:
from sklearn.metrics import classification_report, confusion_matrix

# NOTA: Aquí elegimos analizar un determinado clasificador y sus predicciones 
# Por ejemplo el naive bayes
classifier=mnb_classifier
predictions = mnb_test_predictions
target_names={'Bad','Good'}
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

         Bad       0.78      0.68      0.72       124
        Good       0.72      0.81      0.76       126

    accuracy                           0.74       250
   macro avg       0.75      0.74      0.74       250
weighted avg       0.75      0.74      0.74       250



In [39]:
def print_top20_features_in_trees(vectorizer, clf):
    """Prints features with the highest coefficient values"""
    feature_names = vectorizer.get_feature_names()
    
    top20 = np.argsort(clf.feature_importances_)[-20:]
    reversed_top = top20[::-1]
    print("Top 20 features in the tree\n")
    print("%s" % ( " / ".join(feature_names[j] for j in reversed_top)))

In [40]:
print_top20_features_in_trees(vectorizer,tree_classifier)

Top 20 features in the tree

great / good / amazing / delicious / best / nice / fantastic / awesome / loved / excellent / perfect / friendly / love / happy / enjoyed / wonderful / come / order / fresh / priced


In [43]:
def print_top20_features_per_class_in_NB(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    print("Top 25 features per class\n")
    for i, class_label in enumerate(class_labels):
        top20 = np.argsort(clf.feature_log_prob_[i])[-25:]
        reversed_top = top20[::-1]
        
        print("%s: %s" % (class_label,
              " / ".join(feature_names[j] for j in reversed_top)),'\n')
        
        #Descomentar para ver el índice de los términos en el diccionario
        #print("%s " % (" / ".join(str(j) for j in reversed_top)),'\n')

In [44]:
print_top20_features_per_class_in_NB(vectorizer,mnb_classifier,target_names) 

Top 25 features per class

Bad: food / service / place / won / don / good / like / worst / time / bad / ve / terrible / minutes / disappointed / probably / think / wasn / got / came / just / going / waited / eating / slow / stars 

Good: good / great / food / place / service / delicious / amazing / really / friendly / love / awesome / time / nice / best / fantastic / just / loved / staff / restaurant / experience / like / definitely / atmosphere / happy / excellent 



#### Selecciona el mejor clasificador naive bayes y obtén las 25 variables con más presencia en cada clase:
    
    - ¿Tienen sentido las palabras seleccionadas? ¿hay ruido (palabras sin sentimiento o de sentimiento opuesto al esperado)? ¿por qué crees que suceden estos fenómenos?
    


#### Finalmente, explica de manera razonada las conclusiones que has extraído de todo el estudio realizado en este apartado.






### Apartado b)

#### Toma el mejor clasificador Naive Bayes y el mejor árbol de decisión y analiza a fondo sus resultados en el conjunto de test.

   

#### 1. Analiza la precisión y la exhaustividad de cada clasificador en cada una de las clases (opiniones positivas y negativas).
    
         Para cada clasificador, ¿tiene un comportamiento homogéneo a la hora de clasificar ambas clases?
         ¿Cuáles son las fortalezas y debilidades de cada uno de los clasificadores?
         ¿Hay algún clasificador que sea mejor que el otro en todo?
         ¿Coinciden ambos clasificadores a la hora de clasificar mejor una clase que la otra?
         


#### 2. Pinta los 8 primeros niveles del árbol de decisión y comenta lo que ves.
         ¿Qué estructura tiene el árbol?
         ¿Cómo interpretas los niveles que has pintado? ¿tienen algún sentido con respecto a la tasa de aciertos, o la precisión y exhaustividad del clasificador? o ¿Hay nodos impuros?
         


#### 3. Por cada clasificador identifica 2 críticas que hayan sido falsas positivas (malas críticas calificadas como buenas) y 2 críticas que han sido falsas negativas (buenas críticas clasificadas como malas). Analiza tanto su texto original, como el vector de palabras resultante (solamente los términos activos).
         ¿Por qué crees que ha fallado el clasificador en cada uno de los casos?
         ¿Se te ocurre alguna idea sobre cómo mejorar el clasificador de sentimiento?